# Sensitivity analysis
For research gaps found, we simulate what would have been the mapping of RCTs within regions across diseases if the misclassification of RCTs towards groups of diseases was corrected, given the sensitivities and specificities of the classifier to identify each group of disease.

To estimate the performances of the classifier for each group of diseases, we dispose a test set with 2,763 trials manually classified towards the 27-class grouping of diseases used in this work. The test set is described at Atal et al. BMC Bioinformatics 2016.

The method used is based on the method presented at Fox et al. Int J Epidemiol 2005.

To do so, for each disease for which we found a local research gap we will:

1. Calculating sensitivity and specificity of the classifier to identify the disease and other studies relevant to the burden of diseases, and the number of success and number of trials to derive beta distributions
2. Doing N=60k times the following simulation
    * Randomly choose a sens and spec based on beta distribution for identifying the disease and identifying another disease (no correlation between sens and spec, neither between disease and another disease both)
    * Derive Positive and Negative Predictive Values (PPV and NPV) for each.
    * Simulate the correction of the classification based on PPVs and NPVs
    * Derive the proportion of RCTs concerning the disease among all RCTs concerning the burden of disease in the region
3. Derive 95% upper-bond simulation interval of the proportion of RCTs concerning the disease among all RCTs concerning the burden of diseases

## 1. Sensitivities and specificities based on test set

In [1]:
test_set <- read.table("/media/igna/Elements/HotelDieu/Cochrane/MetaMapBurden/Paper_classifier/NCT_data_classified_to28cats.txt")
dim(test_set)

[1] 2763    8

In [2]:
#We supress injuries from trials concerning the burden of diseases
test_set$GBDnp <- sapply(strsplit(as.character(test_set$GBDnp),"&&"),function(x){paste(x[x!="28"],collapse="&")})
test_set$GBD28 <- sapply(strsplit(as.character(test_set$GBD28),"&"),function(x){paste(x[x!="28"],collapse="&")})

In [3]:
tst <- strsplit(test_set$GBDnp,"&")
alg <- strsplit(test_set$GBD28,"&")
tst <- lapply(tst,as.numeric)
alg <- lapply(alg,as.numeric)

In [4]:
source('Evaluation_metrics.R')

In [5]:
dis <- 1:27
Mgbd <- read.table("/home/igna/Desktop/Programs GBD/Classifier_Trial_GBD/Databases/Taxonomy_DL/GBD_data/GBD_ICD.txt")

In [21]:
#For each category in 1:27, TP, TN, FP and FN of finding the disease and of finding another disease
set.seed(7212)

dis <- as.character(1:27)

PERF_F  <- data.frame()
for(i in dis){
    ALG <- lapply(alg,function(x){rs <- c()
                                  if(i%in%x) rs <- c(1)
                                  if(sum(setdiff(dis,i)%in%x)!=0) rs <- c(rs,2)
                                  return(rs)
                                      })

    DT <- lapply(tst,function(x){rs <- c()
                                if(i%in%x) rs <- c(1)
                                if(sum(setdiff(dis,i)%in%x)!=0) rs <- c(rs,2)
                                return(rs)
                                    })

    CM <- conf_matrix(ALG,DT,c(1,2))

    PERF <- c(CM[1,],CM[2,])
    PERF_F <- rbind(PERF_F,PERF)
}


In [25]:
PERF_F <- data.frame(PERF_F)
names(PERF_F) <- paste(rep(c("TP","FP","TN","FN"),2),rep(c("_Dis","_Oth"),each=4),sep="")

In [27]:
PERF_F$dis <- dis
PERF_F$GBD <- Mgbd$cause_name[-28]

In [29]:
PERF_F <- PERF_F[,c(9,10,1:8)]

In [31]:
write.table(PERF_F,'/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Incertitude_mapping/Sp_Sens_evaluation/Sens_and_Spec_per_27disease_data.txt')

## 2. Simulating correction of misclassification

In [32]:
data <- read.table('/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Flowchart/database_all_diseases_final_ok.txt')

In [33]:
PERF <- read.table('/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Incertitude_mapping/Sp_Sens_evaluation/Sens_and_Spec_per_27disease_data.txt')

In [34]:
GAPS <- read.table('/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Tables/Local_research_gaps.txt')

In [39]:
#For each disease for which we found a gap, we will simulate the mapping across regions of trials concerning
#the disease or concerning other diseases
dis <- levels(GAPS$Disease)

In [40]:
g <- sample(dis,1)

In [43]:
PERF_g <- PERF[PERF$GBD==g,]

In [44]:
sens_r <- PERF_g$TP_dis
sens_n <- PERF_g$TP_dis + PERF_g$FN_dis
spec_r <- PERF_g$TN_dis
spec_n <- PERF_g$TN_dis + PERF_g$FP_dis

In [ ]:
Lreg <- 